### References - Recommender w/ NLP and Graphs

1. [Building a Strong Baseline Recommender in PyTorch, on a Laptop](https://eugeneyan.com/writing/recommender-systems-baseline-pytorch/)
2. [Beating the Baseline Recommender with Graph & NLP in Pytorch](https://eugeneyan.com/writing/recommender-systems-graph-and-nlp-pytorch/)

In [1]:
import argparse
import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score
from torch import optim
from torch.utils.data import DataLoader

In [2]:
from parse_json import parse_json_to_df

In [3]:
shuffle = True
emb_dim = 128
epochs = 5
initial_lr = 0.01

In [4]:
!ls -lah data/

total 11G
drwxrwxr-x 2 roberto roberto 4.0K Oct  8 11:42 .
drwxrwxr-x 5 roberto roberto 4.0K Oct  8 11:53 ..
-rw-rw-r-- 1 roberto roberto 7.7G Oct  8 11:44 review_books.csv
-rw-rw-r-- 1 roberto roberto 3.1G Apr 26  2016 reviews_Books_5.json.gz


In [5]:
#electronics_path = "data/reviews_Electronics_5.json.gz"
#electronics_csv = "data/electronics.csv"
books_path = "data/reviews_Books_5.json.gz"
books_csv = "data/books.csv"

In [6]:
#df = parse_json_to_df(books_path)
#df.tail()
#df.to_csv("data/review_books.csv", index=False)

In [7]:
df = pd.read_csv("data/review_books.csv")

In [8]:
value_reviews = df.overall.value_counts()

In [9]:
value_reviews.values

array([4980815, 2223094,  955189,  415110,  323833])

In [22]:
value_reviews.index.astype('int')

Int64Index([5, 4, 3, 2, 1], dtype='int64')

In [11]:
import altair as alt

In [23]:
ratings = pd.DataFrame({"values": value_reviews.values, "ratings": value_reviews.index.astype('int')})

In [24]:
ratings['values']

0    4980815
1    2223094
2     955189
3     415110
4     323833
Name: values, dtype: int64

In [25]:
ratings['ratings']

0    5
1    4
2    3
3    2
4    1
Name: ratings, dtype: int64

In [47]:
# alt.Chart(ratings).mark_bar().encode(
#     alt.X('ratings:Q',bin=alt.Bin(anchor = 5, extent=[0, 5.5], step=1)),
#     alt.Y('values')
# )
alt.Chart(ratings).mark_bar().encode(
    x='ratings',
    y='values'
)

alt.Chart(...)

In [ ]:
def get_category_lvl(category_list: list, lvl=0) -> str:
    try:
        return category_list[lvl]
    except IndexError:
        return 'NA_VALUE'


def get_categories(df: pd.DataFrame) -> pd.DataFrame:
    df['category_lvl_1'] = df['categories'].apply(get_category_lvl, args=(0,))
    df['category_lvl_2'] = df['categories'].apply(get_category_lvl, args=(1,))
    df['category_lvl_3'] = df['categories'].apply(get_category_lvl, args=(2,))
    df['category_lvl_4'] = df['categories'].apply(get_category_lvl, args=(3,))
    logger.info('Categories lvl 1 - 4 prepared')

    return df


def get_meta(df: pd.DataFrame) -> pd.DataFrame:
    # Update to reflect if relationship exist
    df['related'] = np.where(df['related'].isnull(), 0, 1)

    # Prep categories
    df['categories'] = df['categories'].apply(eval)
    df['categories'] = df['categories'].apply(lambda x: x[0])  # Get first category only
    df = get_categories(df)

    # Prep title and description
    # TODO: Add cleaning of title and description

    return df

In [ ]:
META_COLS = ['asin', 'categories', 'title', 'description', 'price', 'brand', 'related']
df = pd.read_csv(write_books, error_bad_lines=False, warn_bad_lines=True,
                 dtype={'asin': 'str', 'title': 'str', 'brand': 'str'},
                 usecols=META_COLS)
logger.info('DF shape: {}'.format(df.shape))

meta_df = get_meta(df)

meta_df.to_csv(args.write_path, index=False)